In [51]:
from masskg import *
from annotation import *
from iofiles import read_file
from calcformula import get_formula
import os
import pandas as pd
from rdkit import Chem
import pickle
from tqdm import tqdm

In [52]:
param = 'NEG'
pepmass,ms2list,feat = read_file("Source/",f"all_{param.lower()}_with_type.msp",0)
masskg = pd.read_csv("Source/MassKGdatabaseV4.csv")

total ============= 320 blocks finished


/slurm/home/yrd/liaolab/zhubingjie/anaconda3/envs/pyg/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (4,6,7,8,9,10,11,12,13,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
# parameters
with open('model/rt_predict_model.pkl', 'rb') as f:  
   rtmodel = pickle.load(f) 
mode = '-'

# step 1, infer formula

In [54]:
i = 0

formulas = get_formula(mode=mode,mass=pepmass[i],adduct=None)
formulas

,pepmass,Adduct,Formula,ppm,formulascore
0,137.0272,[M-H]-,C5H4N3O2,4.6730,0.646140
1,137.0272,[M-H]-,C7H6O3,3.3300,0.801093
2,137.0272,[M+FA-H]-,C6H4O,3.3300,0.801093
3,137.0272,[M+Cl]-,C2H6N4O,4.1860,0.704370
4,137.0272,[2M-H]-,C3H3NO,-3.9515,0.731771


# step 2, retrieve candidates

In [56]:
# 
for n in range(formulas.shape[0]):
    f =  formulas.iloc[n]['Formula']
    adduct = formulas.iloc[n]['Adduct']
    ppm =  formulas.iloc[n]['ppm']
    formulascore = formulas.iloc[n]['formulascore']
    candi = masskgv4[masskgv3['FORMULA']==f]
    if not len(candi)>0:
        continue
    else:
        candims = [fragments_generation(smiles=candi.iloc[j]['SMILES']) for j in tqdm(range(candi.shape[0]))]
        candi = candi[[len(c)>0 for c in candims]]
        candims = [c for c in candims if len(c)>0]
        candi = candi.reset_index(drop=True)
        candi['adduct'] = adduct
        candi['ppm'] = ppm
        candi['formulascore'] = formulascore
        break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:04<00:00,  8.25it/s]


# step 3, rank candidates    

In [57]:
# rt score
ref_rt = eval(feat['RETENTIONTIME'][i])
mols = [Chem.MolFromSmiles(s) for s in candi['SMILES']]
fps = [np.array(list(MACCSkeys.GenMACCSKeys(mol).ToBitString()),dtype=int) for mol in mols]
pred_rt = rtmodel.predict(fps)
rtscore = [calculate_score(ref_rt,i,0.1) for i in pred_rt]
candi['rtscore'] = rtscore
candi

,InChIKey,SMILES,NAME,FORMULA,Molecular_Weight,Exact_mass,Kingdom,Superclass,Class,Subclass,...,SMILES2,nameTrustLevel,annotationLevel,found_in_databases,taxid,textTaxa,adduct,ppm,formulascore,rtscore
0,ABNPYUDYGCGOTK-UHFFFAOYSA-N,O=C1C(O)=CC=CC=C1O,"2,7-dihydroxycyclohepta-2,4,6-trien-1-one",C7H6O3,138.121011,138.031695,Organic compounds,Hydrocarbon derivatives,Tropones,Tropolones,...,NaN,0.0,1.0,npedia;supernatural2;pubchem,NaN,notax,[M-H]-,3.33,0.801093,2.487968e-17
1,BMBZVGUSOUAIMK-UHFFFAOYSA-N,O=C1C=C(C(=O)C=C1O)C,"2-hydroxy-5-methylcyclohexa-2,5-diene-1,4-dione",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,0.0,1.0,supernatural2;unpd,NaN,notax,[M-H]-,3.33,0.801093,2.375324e-45
2,CCYNZTFDVSWDKP-UHFFFAOYSA-N,O=C1C=CC=C(OC)C1=O,"3-methoxycyclohexa-3,5-diene-1,2-dione",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,0.0,1.0,conmednp;pubchem,NaN,NaN,[M-H]-,3.33,0.801093,2.030187e-45
3,CGINXMWZYDKDBX-UHFFFAOYSA-N,O=C1C=CC(=O)C(=C1O)C,"2-hydroxy-3-methylcyclohexa-2,5-diene-1,4-dione",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,0.0,1.0,supernatural2;unpd;pubchem,NaN,notax,[M-H]-,3.33,0.801093,2.375324e-45
4,CLFRCXCBWIQVRN-UHFFFAOYSA-N,O=CC1=CC(O)=CC=C1O,Gentisaldehyde,C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,0.0,4.0,fooddb;chebi_np;npedia;supernatural2;unpd;nanp...,NaN,notax,[M-H]-,3.33,0.801093,5.721692e-17
5,FJKROLUGYXJWQN-UHFFFAOYSA-N,O=C(O)C1=CC=C(O)C=C1,Paraben,C7H6O3,138.121011,138.031695,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,...,NaN,1.0,5.0,conmednp;fooddb;chemspidernp;tcmdb_taiwan;ibs2...,108544;1037326;304128;237575;136205;4112;65557...,Eriostemon crowei;Begonia nantoensis;Rubus cor...,[M-H]-,3.33,0.801093,1.920073e-05
6,IBGBGRVKPALMCQ-UHFFFAOYSA-N,O=CC1=CC=C(O)C(O)=C1,Catechaldehyde,C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,1.0,5.0,fooddb;chemspidernp;tcmdb_taiwan;ibs2019mar_nc...,54785;1406743;48386;304128;502529;36622;72976;...,Liriodendron tulipifera;Chlorophytum borivilia...,[M-H]-,3.33,0.801093,5.845980e-01
7,IJFXRHURBJZNAO-UHFFFAOYSA-N,O=C(O)C=1C=CC=C(O)C1,3-hydroxybenzoic acid,C7H6O3,138.121011,138.031695,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,...,NaN,2.0,4.0,himdb;phenolexplorer;np_atlas_2019_12;streptom...,153559;153524;49992;141752;228974,Equisetum hiemale;Eukaryota$$$$family;Citrus p...,[M-H]-,3.33,0.801093,1.920073e-05
8,IUNJCFABHJZSKB-UHFFFAOYSA-N,O=CC1=CC=C(O)C=C1O,"2,4-dihydroxybenzaldehyde",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,0.0,4.0,ibs2019mar_nc;supernatural2;knapsack;unpd;pubc...,NaN,Gymnadenia conopsea R.BR.,[M-H]-,3.33,0.801093,1.601326e-24
9,IXWOUPGDGMCKGT-UHFFFAOYSA-N,O=CC=1C=CC=C(O)C1O,"2,3-Dihydroxybenzaldehyde",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,1.0,4.0,cmaup;supernatural2;unpd;npass;pubchem,475932,plants;Citrus wilsonii,[M-H]-,3.33,0.801093,4.622241e-04


In [58]:
# candiscore
candiscore = []
for j in range(candi.shape[0]):
    mol = Chem.MolFromSmiles(candi['SMILES'][j])
    gotms2 = getmatched(ms2list[i],candims[j])
    if len(gotms2)>0:
        predms2 = pred_bde(mol,gotms2,mode=param)
        candiscore.append([predms2['intensity'].sum(),
                           predms2['mes'][0],
                          cosine_similarity(predms2['intensity'].values,1/predms2['bde'].values)])
    else:
         candiscore.append([0,0,0])
candiscore = pd.DataFrame(candiscore)
candiscore.columns = ['structure','mes','cosine']
# meta score
metascore = candi['annotationLevel']
candi = pd.concat([candi,candiscore],axis=1)
candi['score'] = candi['annotationLevel']/5 + 0.1*candi['structure'] + 0.2*candi['mes'] + 0.1*candi['cosine']+candi['rtscore']+candi['formulascore']
candi = candi.sort_values(by='score',ascending=False)

# step 4, return and show annotation results

In [59]:
candi

,InChIKey,SMILES,NAME,FORMULA,Molecular_Weight,Exact_mass,Kingdom,Superclass,Class,Subclass,...,taxid,textTaxa,adduct,ppm,formulascore,rtscore,structure,mes,cosine,score
6,IBGBGRVKPALMCQ-UHFFFAOYSA-N,O=CC1=CC=C(O)C(O)=C1,Catechaldehyde,C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,54785;1406743;48386;304128;502529;36622;72976;...,Liriodendron tulipifera;Chlorophytum borivilia...,[M-H]-,3.33,0.801093,5.845980e-01,0,0,0,2.385691
5,FJKROLUGYXJWQN-UHFFFAOYSA-N,O=C(O)C1=CC=C(O)C=C1,Paraben,C7H6O3,138.121011,138.031695,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,...,108544;1037326;304128;237575;136205;4112;65557...,Eriostemon crowei;Begonia nantoensis;Rubus cor...,[M-H]-,3.33,0.801093,1.920073e-05,0,0,0,1.801112
12,LUSZGTFNYDARNI-UHFFFAOYSA-N,OC1=CC=C2OCOC2=C1,Sesamol,C7H6O3,138.121011,138.031695,Organic compounds,Organoheterocyclic compounds,Benzodioxoles,NaN,...,265764;82096;76098;4182;49511;39353;1140079;10...,Eukaryota$$$$class;plants;Hyptis tomentosa;Ele...,[M-H]-,3.33,0.801093,1.677893e-14,0,0,0,1.801093
14,OUSQDTMHWCQLAU-UHFFFAOYSA-N,O=C1C=C(O)C(=O)C(=C1)C,"3-Hydroxy-2,5-Toluqui颅None",C7H6O3,138.1210106,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,Eukaryota$$$$kingdom,[M-H]-,3.33,0.801093,2.375324e-45,0,0,0,1.801093
17,YGSDEFSMJLZEOE-UHFFFAOYSA-N,O=C(O)C=1C=CC=CC1O,Saligel,C7H6O3,138.121011,138.031695,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,...,1037326;1550870;237575;140420;119176;46220;416...,Utricularia australis;Rubus coreanus;Broussone...,[M-H]-,3.33,0.801093,3.248385e-20,0,0,0,1.801093
9,IXWOUPGDGMCKGT-UHFFFAOYSA-N,O=CC=1C=CC=C(O)C1O,"2,3-Dihydroxybenzaldehyde",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,475932,plants;Citrus wilsonii,[M-H]-,3.33,0.801093,4.622241e-04,0,0,0,1.601555
7,IJFXRHURBJZNAO-UHFFFAOYSA-N,O=C(O)C=1C=CC=C(O)C1,3-hydroxybenzoic acid,C7H6O3,138.121011,138.031695,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,...,153559;153524;49992;141752;228974,Equisetum hiemale;Eukaryota$$$$family;Citrus p...,[M-H]-,3.33,0.801093,1.920073e-05,0,0,0,1.601112
4,CLFRCXCBWIQVRN-UHFFFAOYSA-N,O=CC1=CC(O)=CC=C1O,Gentisaldehyde,C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,notax,[M-H]-,3.33,0.801093,5.721692e-17,0,0,0,1.601093
8,IUNJCFABHJZSKB-UHFFFAOYSA-N,O=CC1=CC=C(O)C=C1O,"2,4-dihydroxybenzaldehyde",C7H6O3,138.121011,138.031695,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Carbonyl compounds,...,NaN,Gymnadenia conopsea R.BR.,[M-H]-,3.33,0.801093,1.601326e-24,0,0,0,1.601093
0,ABNPYUDYGCGOTK-UHFFFAOYSA-N,O=C1C(O)=CC=CC=C1O,"2,7-dihydroxycyclohepta-2,4,6-trien-1-one",C7H6O3,138.121011,138.031695,Organic compounds,Hydrocarbon derivatives,Tropones,Tropolones,...,NaN,notax,[M-H]-,3.33,0.801093,2.487968e-17,0,0,0,1.001093
